In [7]:
%pip install cplex
%pip install docplex

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


In [8]:
from typing import Tuple, List
import logging

def read_hgr(file_path: str) -> Tuple[int, int, List[List[int]]]:
    """
    Read a DIMACS-like .hgr file and return the number of nodes, edges, and the edge list.

    Args:
        file_path: Path to the .hgr file

    Returns:
        Tuple containing:
        - number of nodes
        - number of edges
        - list of edges, where each edge is a list of vertex indices (1-based)

    Example format:
        c I am a comment
        p hs 6 5
        1 2
        2 3 4
        2 4 5
        1 3 6
        5 6
    """
    num_nodes = 0
    num_edges = 0
    edges = []

    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()

            # Skip comments and empty lines
            if not line or line.startswith('c'):
                continue

            # Parse problem line
            if line.startswith('p'):
                parts = line.split()
                if len(parts) != 4 or parts[1] != 'hs':
                    raise ValueError(f"Invalid problem line format: {line}")
                num_nodes = int(parts[2])
                num_edges = int(parts[3])
                continue

            # Parse edge line
            vertices = [int(v) for v in line.split()]
            if not vertices:
                continue

            # Validate vertex numbers
            if any(v < 1 or v > num_nodes for v in vertices):
                raise ValueError(f"Vertex number out of range in line: {line}")

            edges.append(vertices)

    if not num_nodes or not num_edges:
        raise ValueError("Problem line not found or invalid")

    if len(edges) != num_edges:
        logging.warning(f"Expected {num_edges} edges but found {len(edges)}")

    return num_nodes, num_edges, edges


import docplex.mp.model as cplex

def solve_ilp(num_nodes, num_edges, edges, param_dict={}, print_info=False):
    mdl = cplex.Model()

    picked_vertices = mdl.binary_var_list(num_nodes, name="picked_vertices")

    for idx, edge in enumerate(edges):
        mdl.add_constraint(mdl.sum(picked_vertices[node-1] for node in edge) >= 1, "Edge_" + str(idx))

    mdl.minimize(mdl.sum(picked_vertices))

    """Set parameters on the model using a dictionary."""
    for param_path, value in param_dict.items():
        parts = param_path.split('.')
        param = mdl.parameters
        for part in parts:
            param = getattr(param, part)
        param.set(value)

    solution = mdl.solve()
    if print_info:
        mdl.print_information()
        print(mdl.parameters._params)
        print(mdl.solve_details)
        print(f"Solution size: {solution.objective_value}")
        # print(f"Sol: {solution}")
    result = {
        "solve_details": mdl.solve_details,
        "config": param_dict,
        "objective": solution.objective_value if solution else None,
        "solve_status": mdl.solve_details.status,
        "time": mdl.solve_details.time,
        "gap": mdl.solve_details.mip_relative_gap,
        "nodes": mdl.solve_details.nb_nodes_processed,
    }

    return result

In [9]:
FOLDER = 20
import os
if not os.path.exists(f"{FOLDER}"):
	os.mkdir(f"{FOLDER}")
HGRPATH = f"./testset/bremen_subgraph_{FOLDER}.hgr"

num_nodes, num_edges, edges = read_hgr(HGRPATH)
print(f"Graph has {num_nodes} nodes and {num_edges} edges")
solve_ilp(num_nodes, num_edges, edges, print_info=True)

Graph has 32 nodes and 31 edges
Model: docplex_model1922
 - number of variables: 32
   - binary=32, integer=0, continuous=0
 - number of constraints: 31
   - linear=31
 - parameters: defaults
 - objective: minimize
 - problem type is: MILP
[docplex.mp.params.IntParameter(parameters.advance,1), docplex.mp.params.IntParameter(parameters.clocktype,2), docplex.mp.params.NumParameter(parameters.dettimelimit,1e+75), docplex.mp.params.IntParameter(parameters.lpmethod,0), docplex.mp.params.IntParameter(parameters.optimalitytarget,0), docplex.mp.params.IntParameter(parameters.parallel,0), docplex.mp.params.BoolParameter(parameters.paramdisplay,1), docplex.mp.params.IntParameter(parameters.qpmethod,0), docplex.mp.params.IntParameter(parameters.randomseed,202009243), docplex.mp.params.BoolParameter(parameters.record,0), docplex.mp.params.IntParameter(parameters.solutiontype,0), docplex.mp.params.IntParameter(parameters.threads,0), docplex.mp.params.NumParameter(parameters.timelimit,1e+75), docple

{'solve_details': docplex.mp.SolveDetails(time=0.016,status='integer optimal solution'),
 'config': {},
 'objective': 9.0,
 'solve_status': 'integer optimal solution',
 'time': 0.016000000000076398,
 'gap': 0.0,
 'nodes': 0}

In [10]:
from tqdm.contrib.itertools import product
# [docplex.mp.params.IntParameter(parameters.advance,1), docplex.mp.params.IntParameter(parameters.clocktype,2), docplex.mp.params.NumParameter(parameters.dettimelimit,1e+75), docplex.mp.params.IntParameter(parameters.lpmethod,0), docplex.mp.params.IntParameter(parameters.optimalitytarget,0), docplex.mp.params.IntParameter(parameters.parallel,0), docplex.mp.params.BoolParameter(parameters.paramdisplay,1), docplex.mp.params.IntParameter(parameters.qpmethod,0), docplex.mp.params.IntParameter(parameters.randomseed,202009243), docplex.mp.params.BoolParameter(parameters.record,0), docplex.mp.params.IntParameter(parameters.solutiontype,0), docplex.mp.params.IntParameter(parameters.threads,0), docplex.mp.params.NumParameter(parameters.timelimit,1e+75), docplex.mp.params.StrParameter(parameters.workdir,.), docplex.mp.params.NumParameter(parameters.workmem,2048.0)]

import pandas as pd
import numpy as np

NR_RUNS = 30
def gridsearch(param_values, runs_per_data = NR_RUNS):
	rezults = []
	param_names = list(param_values.keys())
	for cfg in product(*param_values.values()):
		# Remove None values (-> defaults)
		cfg = {param_names[i]: v for i, v in enumerate(cfg) if v is not None}
		times = [solve_ilp(num_nodes, num_edges, edges, param_dict=cfg) for _ in range(runs_per_data)]

		rezults +=times
	return rezults

In [11]:
# 138:49:35 HOURS cu toate (cu o singura rulare)
param_values_tols_strat = {
	"threads": [1],
	"mip.tolerances.mipgap":[None, 0.01, 0.05, 0.1],
	"mip.tolerances.integrality":[None, 1e-5, 1e-4, 1e-3],
	"mip.strategy.search": [None, 0, 1, 2],#auto trad dynamic
	# "lpmethod": [None, 1, 2, 3], #primal, dual, barrier
	# "mip.cuts.gomory": [None, 0, 1, 2], #off, moderate, aggressive
	# "mip.cuts.liftproj": [None, 0, 1, 2],
	# "mip.cuts.mircut": [None, 0, 1, 2],
	# "mip.cuts.zerohalfcut": [None, 0, 1, 2],
	# "mip.cuts.flowcovers": [None, 0, 1, 2],
}

# 138:49:35 HOURS cu toate (cu o singura rulare)
rezults_tols_strat = gridsearch(param_values_tols_strat)
rezults_tols_strat = pd.DataFrame(rezults_tols_strat).sort_values(['objective', 'time'], ascending=[True, True])
display(rezults_tols_strat)
rezults_tols_strat.to_json(f"{FOLDER}/rezults_tols_stratt1.json")

  0%|          | 0/64 [00:00<?, ?it/s]

,solve_details,config,objective,solve_status,time,gap,nodes
0,status = integer optimal solution\ntime = ...,{'threads': 1},9.0,integer optimal solution,0.000,0.0,0
2,status = integer optimal solution\ntime = ...,{'threads': 1},9.0,integer optimal solution,0.000,0.0,0
3,status = integer optimal solution\ntime = ...,{'threads': 1},9.0,integer optimal solution,0.000,0.0,0
5,status = integer optimal solution\ntime = ...,{'threads': 1},9.0,integer optimal solution,0.000,0.0,0
6,status = integer optimal solution\ntime = ...,{'threads': 1},9.0,integer optimal solution,0.000,0.0,0
...,...,...,...,...,...,...,...
1882,status = integer optimal solution\ntime = ...,"{'threads': 1, 'mip.tolerances.mipgap': 0.1, '...",9.0,integer optimal solution,0.016,0.0,0
1896,status = integer optimal solution\ntime = ...,"{'threads': 1, 'mip.tolerances.mipgap': 0.1, '...",9.0,integer optimal solution,0.016,0.0,0
1905,status = integer optimal solution\ntime = ...,"{'threads': 1, 'mip.tolerances.mipgap': 0.1, '...",9.0,integer optimal solution,0.016,0.0,0
1916,status = integer optimal solution\ntime = ...,"{'threads': 1, 'mip.tolerances.mipgap': 0.1, '...",9.0,integer optimal solution,0.016,0.0,0


In [12]:
param_values_bnb = {
	"threads": [1],
	"mip.strategy.search": [1],#b&b
	"lpmethod": [1, 2], #primal, dual
	"mip.cuts.gomory": [0, 1, 2], #off, moderate, aggressive
	"mip.cuts.liftproj": [0, 1, 2],
	"mip.cuts.mircut": [0, 1, 2],
	"mip.cuts.zerohalfcut": [0, 1, 2],
}

rezults_bnb = gridsearch(param_values_bnb)
rezults_bnb = pd.DataFrame(rezults_bnb).sort_values(['objective', 'time'], ascending=[True, True])
display(rezults_bnb)
rezults_bnb.to_json(f"{FOLDER}/rezults_bnbt1.json")

  0%|          | 0/162 [00:00<?, ?it/s]

,solve_details,config,objective,solve_status,time,gap,nodes
0,status = integer optimal solution\ntime = ...,"{'threads': 1, 'mip.strategy.search': 1, 'lpme...",9.0,integer optimal solution,0.000,0.0,0
1,status = integer optimal solution\ntime = ...,"{'threads': 1, 'mip.strategy.search': 1, 'lpme...",9.0,integer optimal solution,0.000,0.0,0
3,status = integer optimal solution\ntime = ...,"{'threads': 1, 'mip.strategy.search': 1, 'lpme...",9.0,integer optimal solution,0.000,0.0,0
4,status = integer optimal solution\ntime = ...,"{'threads': 1, 'mip.strategy.search': 1, 'lpme...",9.0,integer optimal solution,0.000,0.0,0
5,status = integer optimal solution\ntime = ...,"{'threads': 1, 'mip.strategy.search': 1, 'lpme...",9.0,integer optimal solution,0.000,0.0,0
...,...,...,...,...,...,...,...
4836,status = integer optimal solution\ntime = ...,"{'threads': 1, 'mip.strategy.search': 1, 'lpme...",9.0,integer optimal solution,0.016,0.0,0
4844,status = integer optimal solution\ntime = ...,"{'threads': 1, 'mip.strategy.search': 1, 'lpme...",9.0,integer optimal solution,0.016,0.0,0
4856,status = integer optimal solution\ntime = ...,"{'threads': 1, 'mip.strategy.search': 1, 'lpme...",9.0,integer optimal solution,0.016,0.0,0
4858,status = integer optimal solution\ntime = ...,"{'threads': 1, 'mip.strategy.search': 1, 'lpme...",9.0,integer optimal solution,0.016,0.0,0


In [13]:
param_values_th = {
	"threads": [None, 1],
}

rezults_th = gridsearch(param_values_th)
rezults_th = pd.DataFrame(rezults_th).sort_values(['objective', 'time'], ascending=[True, True])
display(rezults_th)

  0%|          | 0/2 [00:00<?, ?it/s]

,solve_details,config,objective,solve_status,time,gap,nodes
5,status = integer optimal solution\ntime = ...,{},9.0,integer optimal solution,0.000,0.0,0
24,status = integer optimal solution\ntime = ...,{},9.0,integer optimal solution,0.000,0.0,0
25,status = integer optimal solution\ntime = ...,{},9.0,integer optimal solution,0.000,0.0,0
30,status = integer optimal solution\ntime = ...,{'threads': 1},9.0,integer optimal solution,0.000,0.0,0
31,status = integer optimal solution\ntime = ...,{'threads': 1},9.0,integer optimal solution,0.000,0.0,0
32,status = integer optimal solution\ntime = ...,{'threads': 1},9.0,integer optimal solution,0.000,0.0,0
33,status = integer optimal solution\ntime = ...,{'threads': 1},9.0,integer optimal solution,0.000,0.0,0
34,status = integer optimal solution\ntime = ...,{'threads': 1},9.0,integer optimal solution,0.000,0.0,0
35,status = integer optimal solution\ntime = ...,{'threads': 1},9.0,integer optimal solution,0.000,0.0,0
36,status = integer optimal solution\ntime = ...,{'threads': 1},9.0,integer optimal solution,0.000,0.0,0


In [ ]:
pd.DataFrame(df.groupby('config')['solve_details'])

TypeError: unhashable type: 'dict'

In [19]:
import pandas as pd
FOLDER = 200
# read json into pandas dataframe
df = pd.read_json(f"{FOLDER}/rezults_bnbt1.json")
# Create the time_mean column by grouping by config column and taking the mean of the time attribute of the solve_stats column
df['time_mean'] = df.groupby('config')['solve_status']['time'].transform('mean')

# display the dataframe
display(df)
df.sort_values(['time_mean'], ascending=[True])
print("Max time mean: ", df['time_mean'].max())
# idxmax row
display(df.iloc[df['time_mean'].idxmax()])
print("Min time mean: ", df['time_mean'].min())
display(df.iloc[df['time_mean'].idxmin()])
print("Mean time mean: ", df['time_mean'].mean())
print("Median time mean: ", df['time_mean'].median())

IndexError: Column(s) solve_status already selected